# 🔍 Apache Hive SQL Operations

This notebook demonstrates Hive SQL operations with PostgreSQL metastore integration.

## Learning Objectives
- Connect to Hive with PostgreSQL metastore
- Create databases and tables
- Load data into Hive tables
- Perform SQL queries and analytics
- Work with partitioned tables

## 1. Hive Setup and Connection

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd

print('🚀 Big Data Environment - Apache Hive SQL Operations')
print('=' * 60)

# Create Spark session with Hive support
spark = SparkSession.builder \
    .appName("BigDataEnv-HiveSQL") \
    .master("spark://spark-master:7077") \
    .config("spark.sql.warehouse.dir", "hdfs://namenode:9000/user/hive/warehouse") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000") \
    .config("spark.sql.catalogImplementation", "hive") \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()

print('✅ Spark Session with Hive support created')
print(f'🗄️  Warehouse Directory: hdfs://namenode:9000/user/hive/warehouse')
print(f'🔗 Metastore URI: thrift://hive-metastore:9083')

# Test Hive connection
try:
    databases = spark.sql("SHOW DATABASES")
    print('\n📋 Available Databases:')
    databases.show()
except Exception as e:
    print(f'⚠️  Hive connection issue: {e}')
    print('💡 Continuing with Spark SQL (Hive compatibility mode)')

## 2. Create Database and Tables

In [ ]:
# Create a new database
print('🏗️  Creating Hive Database and Tables:')

try:
    # Create database
    spark.sql("CREATE DATABASE IF NOT EXISTS bigdata_demo")
    spark.sql("USE bigdata_demo")
    print('✅ Database "bigdata_demo" created and selected')
    
    # Show current database
    current_db = spark.sql("SELECT current_database()")
    current_db.show()
    
except Exception as e:
    print(f'❌ Error creating database: {e}')
    print('💡 Using default database')

In [ ]:
# Create users table
print('👥 Creating Users Table:')

create_users_table = """
CREATE TABLE IF NOT EXISTS users (
    user_id INT,
    name STRING,
    email STRING,
    age INT,
    city STRING,
    country STRING
)
STORED AS PARQUET
LOCATION 'hdfs://namenode:9000/user/hive/warehouse/bigdata_demo.db/users'
"""

try:
    spark.sql(create_users_table)
    print('✅ Users table created successfully')
except Exception as e:
    print(f'❌ Error creating users table: {e}')

# Create transactions table
print('\n💳 Creating Transactions Table:')

create_transactions_table = """
CREATE TABLE IF NOT EXISTS transactions (
    transaction_id STRING,
    user_id INT,
    amount DOUBLE,
    currency STRING,
    merchant STRING,
    category STRING,
    status STRING,
    transaction_date DATE
)
PARTITIONED BY (year INT, month INT)
STORED AS PARQUET
LOCATION 'hdfs://namenode:9000/user/hive/warehouse/bigdata_demo.db/transactions'
"""

try:
    spark.sql(create_transactions_table)
    print('✅ Transactions table created successfully (partitioned)')
except Exception as e:
    print(f'❌ Error creating transactions table: {e}')

## 3. Load Data into Hive Tables

In [ ]:
# Load users data from HDFS CSV file
print('📤 Loading Users Data:')

try:
    # Read users CSV from HDFS
    users_df = spark.read \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .csv("hdfs://namenode:9000/user/demo/input/users.csv")
    
    # Insert into Hive table
    users_df.write \
        .mode("overwrite") \
        .insertInto("users")
    
    print('✅ Users data loaded into Hive table')
    
    # Verify data
    user_count = spark.sql("SELECT COUNT(*) as count FROM users")
    user_count.show()
    
except Exception as e:
    print(f'❌ Error loading users data: {e}')
    # Create sample data if file loading fails
    sample_users = [
        (1, "John Smith", "john@email.com", 28, "New York", "USA"),
        (2, "Emma Johnson", "emma@email.com", 34, "London", "UK"),
        (3, "Michael Chen", "michael@email.com", 22, "Toronto", "Canada"),
        (4, "Sarah Williams", "sarah@email.com", 31, "Sydney", "Australia"),
        (5, "David Brown", "david@email.com", 45, "Berlin", "Germany")
    ]
    columns = ["user_id", "name", "email", "age", "city", "country"]
    sample_df = spark.createDataFrame(sample_users, columns)
    sample_df.createOrReplaceTempView("users")
    print('ℹ️  Using sample users data in temporary view')

In [ ]:
# Load and process transactions data
print('💳 Loading Transactions Data:')

try:
    # Read transactions JSON from HDFS
    transactions_raw = spark.read \
        .option("multiline", "true") \
        .json("hdfs://namenode:9000/user/demo/input/transactions.json")
    
    # Process and add partitioning columns
    transactions_processed = transactions_raw \
        .withColumn("transaction_date", to_date(col("timestamp"))) \
        .withColumn("year", year(to_date(col("timestamp")))) \
        .withColumn("month", month(to_date(col("timestamp")))) \
        .select(
            "transaction_id", "user_id", "amount", "currency", 
            "merchant", "category", "status", "transaction_date", 
            "year", "month"
        )
    
    # Show sample processed data
    print('\n🔍 Sample Processed Transactions:')
    transactions_processed.show(5)
    
    # Create temporary view for SQL operations
    transactions_processed.createOrReplaceTempView("transactions")
    print('✅ Transactions data processed and available for queries')
    
except Exception as e:
    print(f'❌ Error loading transactions: {e}')
    # Create sample transactions data
    sample_transactions = [
        ("TXN001", 1, 150.50, "USD", "Amazon", "Electronics", "completed", "2024-01-15", 2024, 1),
        ("TXN002", 2, 89.99, "GBP", "Tesco", "Groceries", "completed", "2024-01-15", 2024, 1),
        ("TXN003", 3, 299.00, "CAD", "Best Buy", "Electronics", "pending", "2024-01-15", 2024, 1),
        ("TXN004", 1, 45.75, "USD", "Starbucks", "Food & Drink", "completed", "2024-01-16", 2024, 1),
        ("TXN005", 4, 120.00, "AUD", "Woolworths", "Groceries", "completed", "2024-01-16", 2024, 1)
    ]
    columns = ["transaction_id", "user_id", "amount", "currency", "merchant", "category", "status", "transaction_date", "year", "month"]
    sample_trans_df = spark.createDataFrame(sample_transactions, columns)
    sample_trans_df.createOrReplaceTempView("transactions")
    print('ℹ️  Using sample transactions data in temporary view')

## 4. Basic Hive SQL Queries

In [ ]:
# Basic SELECT queries
print('🔍 Basic Hive SQL Queries:')

# Query 1: Show all users
print('\n👥 All Users:')
all_users = spark.sql("""
    SELECT user_id, name, email, age, city, country 
    FROM users 
    ORDER BY user_id
""")
all_users.show()

# Query 2: Users by country with statistics
print('\n🌍 Users by Country:')
users_by_country = spark.sql("""
    SELECT 
        country,
        COUNT(*) as user_count,
        AVG(age) as avg_age,
        MIN(age) as min_age,
        MAX(age) as max_age
    FROM users 
    GROUP BY country 
    ORDER BY user_count DESC
""")
users_by_country.show()

# Query 3: Users in specific age range
print('\n🎯 Users between 25 and 35:')
age_range_users = spark.sql("""
    SELECT name, age, city, country 
    FROM users 
    WHERE age BETWEEN 25 AND 35 
    ORDER BY age
""")
age_range_users.show()

In [ ]:
# Transaction analysis queries
print('💰 Transaction Analysis Queries:')

# Query 1: Transaction summary by status
print('\n📊 Transactions by Status:')
transactions_by_status = spark.sql("""
    SELECT 
        status,
        COUNT(*) as transaction_count,
        SUM(amount) as total_amount,
        AVG(amount) as avg_amount
    FROM transactions 
    GROUP BY status 
    ORDER BY transaction_count DESC
""")
transactions_by_status.show()

# Query 2: Top merchants by revenue
print('\n🏪 Top Merchants by Revenue:')
top_merchants = spark.sql("""
    SELECT 
        merchant,
        COUNT(*) as transaction_count,
        SUM(amount) as total_revenue,
        AVG(amount) as avg_transaction_value
    FROM transactions 
    WHERE status = 'completed'
    GROUP BY merchant 
    ORDER BY total_revenue DESC
""")
top_merchants.show()

# Query 3: Transactions by category
print('\n📦 Transactions by Category:')
category_analysis = spark.sql("""
    SELECT 
        category,
        COUNT(*) as transaction_count,
        SUM(amount) as total_spent,
        ROUND(AVG(amount), 2) as avg_amount
    FROM transactions 
    WHERE status = 'completed'
    GROUP BY category 
    ORDER BY total_spent DESC
""")
category_analysis.show()

## 5. Advanced Hive SQL - JOINs and Subqueries

In [ ]:
# Complex JOIN queries
print('🔗 Advanced JOIN Queries:')

# Query 1: User spending analysis with JOIN
print('\n💸 User Spending Analysis:')
user_spending = spark.sql("""
    SELECT 
        u.user_id,
        u.name,
        u.country,
        u.age,
        COUNT(t.transaction_id) as total_transactions,
        SUM(CASE WHEN t.status = 'completed' THEN t.amount ELSE 0 END) as total_spent,
        SUM(CASE WHEN t.status = 'pending' THEN t.amount ELSE 0 END) as pending_amount,
        SUM(CASE WHEN t.status = 'failed' THEN t.amount ELSE 0 END) as failed_amount,
        ROUND(AVG(t.amount), 2) as avg_transaction_amount
    FROM users u
    LEFT JOIN transactions t ON u.user_id = t.user_id
    GROUP BY u.user_id, u.name, u.country, u.age
    ORDER BY total_spent DESC
""")
user_spending.show()

# Query 2: Country-wise spending patterns
print('\n🌍 Country-wise Spending Patterns:')
country_spending = spark.sql("""
    SELECT 
        u.country,
        COUNT(DISTINCT u.user_id) as active_users,
        COUNT(t.transaction_id) as total_transactions,
        SUM(t.amount) as total_revenue,
        ROUND(AVG(t.amount), 2) as avg_transaction_value,
        ROUND(SUM(t.amount) / COUNT(DISTINCT u.user_id), 2) as revenue_per_user
    FROM users u
    JOIN transactions t ON u.user_id = t.user_id
    WHERE t.status = 'completed'
    GROUP BY u.country
    ORDER BY total_revenue DESC
""")
country_spending.show()

In [ ]:
# Advanced analytics with window functions
print('📊 Advanced Analytics with Window Functions:')

# Query 1: User ranking by spending
print('\n🏆 User Ranking by Total Spending:')
user_ranking = spark.sql("""
    SELECT 
        u.name,
        u.country,
        SUM(t.amount) as total_spent,
        RANK() OVER (ORDER BY SUM(t.amount) DESC) as spending_rank,
        DENSE_RANK() OVER (PARTITION BY u.country ORDER BY SUM(t.amount) DESC) as country_rank
    FROM users u
    JOIN transactions t ON u.user_id = t.user_id
    WHERE t.status = 'completed'
    GROUP BY u.user_id, u.name, u.country
    ORDER BY total_spent DESC
""")
user_ranking.show()

# Query 2: Running totals and percentages
print('\n📈 Running Totals and Cumulative Analysis:')
running_totals = spark.sql("""
    SELECT 
        merchant,
        SUM(amount) as merchant_revenue,
        SUM(SUM(amount)) OVER (ORDER BY SUM(amount) DESC 
                              ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as running_total,
        ROUND(SUM(amount) * 100.0 / SUM(SUM(amount)) OVER (), 2) as pct_of_total
    FROM transactions 
    WHERE status = 'completed'
    GROUP BY merchant
    ORDER BY merchant_revenue DESC
""")
running_totals.show()

## 6. Data Export and Table Management

In [ ]:
# Create summary tables
print('💾 Creating Summary Tables:')

# Create user summary table
print('\n👥 Creating User Summary Table:')
create_user_summary = spark.sql("""
    CREATE TABLE IF NOT EXISTS user_summary AS
    SELECT 
        u.user_id,
        u.name,
        u.country,
        u.age,
        COUNT(t.transaction_id) as total_transactions,
        SUM(CASE WHEN t.status = 'completed' THEN t.amount ELSE 0 END) as total_spent,
        ROUND(AVG(CASE WHEN t.status = 'completed' THEN t.amount END), 2) as avg_transaction
    FROM users u
    LEFT JOIN transactions t ON u.user_id = t.user_id
    GROUP BY u.user_id, u.name, u.country, u.age
""")

print('✅ User summary table created')

# Show the summary table
print('\n📊 User Summary Table Contents:')
spark.sql("SELECT * FROM user_summary ORDER BY total_spent DESC").show()

# Create merchant performance table
print('\n🏪 Creating Merchant Performance Table:')
merchant_performance = spark.sql("""
    CREATE TABLE IF NOT EXISTS merchant_performance AS
    SELECT 
        merchant,
        category,
        COUNT(*) as transaction_count,
        SUM(amount) as total_revenue,
        ROUND(AVG(amount), 2) as avg_transaction_value,
        COUNT(CASE WHEN status = 'completed' THEN 1 END) as successful_transactions,
        COUNT(CASE WHEN status = 'failed' THEN 1 END) as failed_transactions,
        ROUND(COUNT(CASE WHEN status = 'completed' THEN 1 END) * 100.0 / COUNT(*), 2) as success_rate
    FROM transactions
    GROUP BY merchant, category
""")

print('✅ Merchant performance table created')
print('\n📊 Merchant Performance:')
spark.sql("SELECT * FROM merchant_performance ORDER BY total_revenue DESC").show()

In [ ]:
# Export data to different formats
print('📤 Exporting Data to HDFS:')

try:
    # Export user summary as Parquet
    user_summary_df = spark.sql("SELECT * FROM user_summary")
    user_summary_df.coalesce(1) \
        .write \
        .mode("overwrite") \
        .parquet("hdfs://namenode:9000/user/demo/output/hive_user_summary")
    print('✅ User summary exported as Parquet')
    
    # Export merchant performance as JSON
    merchant_perf_df = spark.sql("SELECT * FROM merchant_performance")
    merchant_perf_df.coalesce(1) \
        .write \
        .mode("overwrite") \
        .json("hdfs://namenode:9000/user/demo/output/hive_merchant_performance")
    print('✅ Merchant performance exported as JSON')
    
    # Export detailed analysis as CSV
    detailed_analysis = spark.sql("""
        SELECT 
            u.name,
            u.country,
            t.merchant,
            t.category,
            t.amount,
            t.currency,
            t.status,
            t.transaction_date
        FROM users u
        JOIN transactions t ON u.user_id = t.user_id
        ORDER BY t.transaction_date, u.name
    """)
    
    detailed_analysis.coalesce(1) \
        .write \
        .mode("overwrite") \
        .option("header", "true") \
        .csv("hdfs://namenode:9000/user/demo/output/hive_detailed_analysis")
    print('✅ Detailed analysis exported as CSV')
    
except Exception as e:
    print(f'❌ Error exporting data: {e}')

## 7. Table Information and Metadata

In [ ]:
# Show table information
print('ℹ️  Table Information and Metadata:')

# List all tables
print('\n📋 Available Tables:')
try:
    tables = spark.sql("SHOW TABLES")
    tables.show()
except:
    print('📊 Temporary views:')
    spark.catalog.listTables().show()

# Describe table structure
print('\n🔍 Users Table Structure:')
try:
    spark.sql("DESCRIBE EXTENDED users").show(20, False)
except:
    spark.sql("DESCRIBE users").show()

# Show table statistics
print('\n📊 Table Statistics:')
try:
    # Analyze table to compute statistics
    spark.sql("ANALYZE TABLE users COMPUTE STATISTICS")
    spark.sql("SHOW TBLPROPERTIES users").show(10, False)
except Exception as e:
    print(f'ℹ️  Statistics not available: {e}')
    
# Show table size and location
print('\n💾 Table Storage Information:')
users_count = spark.sql("SELECT COUNT(*) as total_users FROM users")
transactions_count = spark.sql("SELECT COUNT(*) as total_transactions FROM transactions")

print('Table sizes:')
users_count.show()
transactions_count.show()

## 8. Performance Optimization

In [ ]:
# Performance optimization techniques
print('⚡ Performance Optimization:')

# Query with explain plan
print('\n🔍 Query Execution Plan:')
complex_query = spark.sql("""
    SELECT 
        u.country,
        COUNT(*) as user_count,
        SUM(t.amount) as total_revenue
    FROM users u
    JOIN transactions t ON u.user_id = t.user_id
    WHERE t.status = 'completed'
    GROUP BY u.country
    ORDER BY total_revenue DESC
""")

# Show execution plan
complex_query.explain(True)

# Cache frequently used table
print('\n💾 Caching Frequently Used Tables:')
spark.sql("CACHE TABLE users")
print('✅ Users table cached in memory')

# Show cached tables
try:
    cached_tables = spark.sql("SHOW TABLES")
    print('\n📋 Cached Tables:')
    cached_tables.show()
except:
    print('💡 Cache information not available via SQL')

print('\n💡 Performance Tips:')
print('  - Use partitioned tables for large datasets')
print('  - Cache frequently accessed tables')
print('  - Use appropriate file formats (Parquet, ORC)')
print('  - Optimize JOIN order and conditions')
print('  - Use columnar storage for analytics workloads')

## 9. Cleanup and Best Practices

In [ ]:
# Cleanup operations
print('🧹 Cleanup and Best Practices:')

# Uncache tables
print('\n💾 Uncaching Tables:')
try:
    spark.sql("UNCACHE TABLE users")
    print('✅ Users table removed from cache')
except:
    print('ℹ️  No cached tables to remove')

# Show current database and tables
print('\n📋 Current Database Status:')
try:
    current_db = spark.sql("SELECT current_database()")
    current_db.show()
    
    tables = spark.sql("SHOW TABLES")
    print('\nTables in current database:')
    tables.show()
except:
    print('ℹ️  Database information not available')

print('\n💡 Hive Best Practices:')
best_practices = [
    "1. Use appropriate data types for better performance",
    "2. Partition tables based on query patterns",
    "3. Use bucketing for frequently joined columns",
    "4. Choose optimal file formats (Parquet for analytics)",
    "5. Regularly analyze tables to update statistics",
    "6. Use external tables for data flexibility",
    "7. Implement proper data lifecycle management",
    "8. Monitor and optimize query performance"
]

for practice in best_practices:
    print(f'   {practice}')

print('\n🔗 Useful Resources:')
print('   - HiveServer2 UI: http://localhost:10002')
print('   - Spark SQL UI: http://localhost:4040')
print('   - HDFS NameNode: http://localhost:9870')

## 🎯 Summary

In this notebook, you learned:

1. **Hive Setup**: Connecting to Hive with PostgreSQL metastore
2. **Database Management**: Creating databases and tables with proper schemas
3. **Data Loading**: Loading data from various sources into Hive tables
4. **SQL Queries**: Basic and advanced SQL operations in Hive
5. **JOINs and Analytics**: Complex queries with joins and window functions
6. **Data Export**: Exporting results to different formats and locations
7. **Metadata Management**: Working with table information and statistics
8. **Performance Optimization**: Query optimization and caching strategies

### Key Hive Concepts Covered
- **DDL Operations**: `CREATE DATABASE`, `CREATE TABLE`, `DESCRIBE`
- **DML Operations**: `INSERT`, `SELECT`, `JOIN`, `GROUP BY`
- **Analytics Functions**: `RANK()`, `DENSE_RANK()`, Window functions
- **Data Formats**: Parquet, JSON, CSV storage options
- **Partitioning**: Table partitioning for better performance

### Integration Benefits
- **PostgreSQL Metastore**: Reliable metadata storage (no Derby issues)
- **Spark Integration**: Fast query execution with Spark engine
- **HDFS Storage**: Scalable distributed file system
- **Schema Evolution**: Flexible schema management

### Next Steps
- Explore the **04-integration.ipynb** notebook for full stack integration
- Check out the HiveServer2 UI at http://localhost:10002
- Browse HDFS files at http://localhost:9870

### 🔗 Useful Links
- **HiveServer2 UI**: http://localhost:10002
- **Spark Master UI**: http://localhost:8080
- **HDFS NameNode UI**: http://localhost:9870
- **Hive Documentation**: https://hive.apache.org/
- **HiveQL Reference**: https://cwiki.apache.org/confluence/display/Hive/LanguageManual